<a href="https://colab.research.google.com/github/DenisR144/healthdata22/blob/main/Dataton_2022_(DR_and_T_SH).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Основные действия по основной информации**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Подключение основных библиотек**
ЗАПУСКАТЬ ВСЕМ!!!

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db
#import re
!pip install pymorphy2
import pymorphy2
import pymystem3
from collections import Counter
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 8.2 MB 33.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=b3009deac95fec1f6b312464e1bdad12b9b83f9e5ebea770a74eb7d6f4f5bc4e
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


**Создание основной БД (без текстовых полей)** ( НЕ ЗАПУСКАТЬ!!!)

In [ ]:
#UID;Date;Name;DOB;MKB_1;MKB_2;MKB_3;Died;Violations;ViolationsCausedDeath;Hospital;Ambulance;Home;Meddoc
#df = pd.read_csv("/content/drive/MyDrive/DataMed.csv")
df = pd.read_csv("DataMed.csv")
print(df)

   UID;Date;Name;DOB;MKB_1;MKB_2;MKB_3;Died;Violations;ViolationsCausedDeath;Hospital;Ambulance;Home;Meddoc
0   1;15.03.2022;;21.07.2021;C;61;2;1;0;0;1;0;1;1.txt                                                      
1   2;21.02.2022;;20.07.2018;Î;26;8;1;0;0;1;1;0;2.txt                                                      
2   3;01.12.2021;;08.07.1929;I;63;0;1;1;0;1;0;0;3.txt                                                      
3   4;28.10.2021;;22.12.1983;G;40;3;1;0;0;1;0;0;4.txt                                                      
4   5;26.07.2021;;31.10.2007;Ñ;73;0;1;0;0;1;0;0;5.txt                                                      
5   6;11.10.2021;;04.10.2007;Q;24;8;0;0;0;1;0;0;6.txt                                                      
6   7;29.06.2021;;08.04.1949;I;69;3;1;0;0;1;0;0;7.txt                                                      
7   8;28.04.2021;;13.08.1979;M;79;1;1;0;0;1;0;0;8.txt                                                      
8   9;14.04.2021;;31.10.1928

In [ ]:
def open_csv_file(csv_file_path):
    """
    Open and read data from a csv file without headers (skipping the first row)
    :param csv_file_path: path of the csv file to process
    :return: a list with the csv content
    """
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)

        data = list()
        for row in reader:
            data.append(row)

        return data


In [ ]:

#conn = db.connect(':memory:')
conn = db.connect('dataton_db.db')
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS table1")

create_table = """CREATE TABLE table1(
                uid INTEGER PRIMARY KEY AUTOINCREMENT,
                date DATE NOT NULL,
                name VARCHAR(10) NOT NULL,
                dob DATE NOT NULL,
                mkb1 VARCHAR(1) NOT NULL,
                mkb2 INTEGER NOT NULL,
                mkb3 INTEGER NOT NULL,
                died INTEGER NOT NULL,
                violations INTEGER NOT NULL,
                vcd INTEGER NOT NULL,
                hospital INTEGER NOT NULL,
                ambulance INTEGER NOT NULL,
                home INTEGER NOT NULL,
                meddoc VARCHAR(50) NOT NULL,
                perc_of_noun FLOAT64 NULL,
                perc_of_adj FLOAT64 NULL,
                perc_of_verb FLOAT64 NULL,
                alcohol_num INT NULL,
                tabacco_num INT NULL
                );"""
 
cur.execute(create_table)

#values_to_insert = open_csv_file('drive/MyDrive/DataMed.csv')
values_to_insert = open_csv_file('DataMed.csv')
for vv in values_to_insert:
  insert_records = """INSERT INTO table1(uid,date,name,dob,mkb1,mkb2,mkb3,died,violations,vcd,hospital,ambulance,home,meddoc) 
  VALUES(
    """+vv[0].split(';')[0]+""",
    """+vv[0].split(';')[1]+""",
    '"""+vv[0].split(';')[2]+"""',
    """+vv[0].split(';')[3]+""",
    '"""+vv[0].split(';')[4]+"""',
    """+vv[0].split(';')[5]+""",
    """+vv[0].split(';')[6]+""",
    """+vv[0].split(';')[7]+""",
    """+vv[0].split(';')[8]+""",
    """+vv[0].split(';')[9]+""",
    """+vv[0].split(';')[10]+""",
    """+vv[0].split(';')[11]+""",
    """+vv[0].split(';')[12]+""",
    '"""+vv[0].split(';')[13]+"""'
    )
  """
  cur.execute(insert_records)
  #print(insert_records)




**Подгрузка текстов** (НЕ ЗАПУСКАТЬ!!!)

In [ ]:
def parse_meddoc(filename : str): #-> perc_of_noun,perc_of_adj,perc_of_verb,alcohol_num,tabacco_num
  myfile = open(filename,'r') 
  words = []

  try:
    filelines = myfile.readlines()
    for line in filelines:
      words += line.split(' ')

  finally:
    pass
    #f.close()

  words2 = words
  words3 = []

  # 1 - удаление специальных символов

  #reg = re.compile('[^a-zA-Z]')
  n = 0
  for val in words2:
    val = val.replace(" ","")
    val = val.replace(".","")
    val = val.replace(",","")
    val = val.replace(":","")
    val = val.replace(";","")
    val = val.replace("1","")
    val = val.replace("2","")
    val = val.replace("3","")
    val = val.replace("4","")
    val = val.replace("5","")
    val = val.replace("6","")
    val = val.replace("7","")
    val = val.replace("8","")
    val = val.replace("9","")
    val = val.replace("0","")
    val = val.replace("(","")
    val = val.replace(")","")
    if len(val)>1: #отсекаем пустые сроки, и односимвольные
      words3.append(val)

  print("Strings are affected")



  # Лемматизация
  # Рабочий массив данных - список words3

  words5 = words3
  words6 = []
  words_pos = []

  m1 = pymorphy2.MorphAnalyzer()

  n=0
  for val in words5:
    p = m1.parse(val)[0]
    words6.append(p.normal_form)
    words_pos.append(p.tag.POS)
    n+=1

  print(str(n) + " words affected!")


  # считаем количество слов по частям речи

  c = Counter(words_pos)
  
  perc_of_noun = c['NOUN'] / len(words_pos) # доля существительных
  perc_of_adj = c['ADJF'] / len(words_pos) # доля прилагательных
  perc_of_verb = c['VERB'] / len(words_pos) # доля глаголов

  # считаем количество слов по гипотезам (табак, алкоголь)

  c2 = Counter(words6)
  alcohol_num = c2['спирт'] + c2['выпивка'] + c2['алкоголь'] + c2['выпивать'] + c2['выпить'] + c2['пить'] + c2['водка'] + c2['пиво'] + c2['употреблять']
  tabacco_num = c2['табак'] + c2['сигарета'] + c2['курить'] + c2['выкуривать']

  return(perc_of_noun,perc_of_adj,perc_of_verb,alcohol_num,tabacco_num)



**Обработка текстов**

1) Стемминг (отрезаем в словах окончания и суффиксы), есть вроде готовые библиотеки под русский язык //минусы - теряем часть информации, а часть - сливается

*или* 

2) Лемматизация (морфологический анализ) с целью логического объединения разных форм (например "шёл", "идти" - перевод в слов в начальные формы) - библиотеки pymorphy2, pymystem3

3) частотный анализ, т.е. закидываем все слова с частотным анализом в матрицу, и дальше обрабатываем как вектора
Это надо делать после выше приведенных 1 или 2

4) составление матрицы по частоте использования частей речи (глагол, существительное, прилагательное), перевод их в процентное соотношение от объема текста

1. избавимся от всех лишних символов, кроме русских букв, дефисов и пробелов.


2. цепочка дат событий: приведение дат к единому формату, выстраивание цепочек

3. создание словаря - значимых терминов (от Дениса Рощина, максимальная вероятность получить нормальный словарь)
далее слова из этого словаря найти в текстах и по каждой истории болезни вывести частотную характеристику по каждому слову словаря
В словаре могут быть синонимы.
Статистику по синонимам объединить.

Со словарем значимых слов мне кажется будет решение изящнее...

In [31]:
res = cur.execute("SELECT uid,meddoc FROM table1")
row = res.fetchall()
row

for a in row:
  aa = parse_meddoc(a[1]) #perc_of_noun,perc_of_adj,perc_of_verb,alcohol_num,tabacco_num
  res2 = cur.execute("UPDATE table1 SET perc_of_noun = '"+str(aa[0])+"', perc_of_adj = '"+str(aa[1])+"',perc_of_verb = '"+str(aa[2])+"',alcohol_num = '"+str(aa[3])+"',tabacco_num = '"+str(aa[4])+"' WHERE UID="+str(a[0]))

res = cur.execute("SELECT * FROM table1")
row = res.fetchall()
row

[(1,
  2004,
  '',
  1993,
  'C',
  61,
  2,
  1,
  0,
  0,
  1,
  0,
  1,
  '1.txt',
  0.5193392806481102,
  0.21375658039907952,
  0.03574693440090786,
  0,
  0),
 (2,
  1999,
  '',
  1991,
  'I',
  26,
  8,
  1,
  0,
  0,
  1,
  1,
  0,
  '2.txt',
  0.49448605295748066,
  0.19567140413988324,
  0.04680466277446876,
  5,
  0),
 (3,
  2008,
  '',
  1914,
  'I',
  63,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  '3.txt',
  0.4752530933633296,
  0.2116610423697038,
  0.03880764904386952,
  0,
  0),
 (4,
  1983,
  '',
  1949,
  'G',
  40,
  3,
  1,
  0,
  0,
  1,
  0,
  0,
  '4.txt',
  0.5254506892895016,
  0.19628844114528102,
  0.03361611876988335,
  0,
  0),
 (5,
  1988,
  '',
  1966,
  'N',
  73,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  '5.txt',
  0.49965641642329495,
  0.20889881463666038,
  0.035131420718089675,
  0,
  0),
 (6,
  2000,
  '',
  1993,
  'Q',
  24,
  8,
  0,
  0,
  0,
  1,
  0,
  0,
  '6.txt',
  0.5212983079860425,
  0.1811179142800711,
  0.05128711567581803,
  15,
  3),
 (7,
  1

In [32]:
res = cur.execute("PRAGMA table_info(table1);")
row = res.fetchall()

row

conn.commit()
conn.close()

**Основной анализ обработанного текста и основных полей**

ЗАПУСКАТЬ ВСЕМ

In [33]:
conn = db.connect('dataton_db.db')
cur = conn.cursor()
res = cur.execute("PRAGMA table_info(table1);")
row = res.fetchall()


row

[(0, 'uid', 'INTEGER', 0, None, 1),
 (1, 'date', 'DATE', 1, None, 0),
 (2, 'name', 'VARCHAR(10)', 1, None, 0),
 (3, 'dob', 'DATE', 1, None, 0),
 (4, 'mkb1', 'VARCHAR(1)', 1, None, 0),
 (5, 'mkb2', 'INTEGER', 1, None, 0),
 (6, 'mkb3', 'INTEGER', 1, None, 0),
 (7, 'died', 'INTEGER', 1, None, 0),
 (8, 'violations', 'INTEGER', 1, None, 0),
 (9, 'vcd', 'INTEGER', 1, None, 0),
 (10, 'hospital', 'INTEGER', 1, None, 0),
 (11, 'ambulance', 'INTEGER', 1, None, 0),
 (12, 'home', 'INTEGER', 1, None, 0),
 (13, 'meddoc', 'VARCHAR(50)', 1, None, 0),
 (14, 'perc_of_noun', 'FLOAT64', 0, None, 0),
 (15, 'perc_of_adj', 'FLOAT64', 0, None, 0),
 (16, 'perc_of_verb', 'FLOAT64', 0, None, 0),
 (17, 'alcohol_num', 'INT', 0, None, 0),
 (18, 'tabacco_num', 'INT', 0, None, 0)]

In [37]:
df = pd.read_sql("SELECT * FROM table1", conn)

In [38]:
df

,uid,date,name,dob,mkb1,mkb2,mkb3,died,violations,vcd,hospital,ambulance,home,meddoc,perc_of_noun,perc_of_adj,perc_of_verb,alcohol_num,tabacco_num
0,1,2004,,1993,C,61,2,1,0,0,1,0,1,1.txt,0.519339,0.213757,0.035747,0,0
1,2,1999,,1991,I,26,8,1,0,0,1,1,0,2.txt,0.494486,0.195671,0.046805,5,0
2,3,2008,,1914,I,63,0,1,1,0,1,0,0,3.txt,0.475253,0.211661,0.038808,0,0
3,4,1983,,1949,G,40,3,1,0,0,1,0,0,4.txt,0.525451,0.196288,0.033616,0,0
4,5,1988,,1966,N,73,0,1,0,0,1,0,0,5.txt,0.499656,0.208899,0.035131,0,0
5,6,2000,,1993,Q,24,8,0,0,0,1,0,0,6.txt,0.521298,0.181118,0.051287,15,3
6,7,1986,,1937,I,69,3,1,0,0,1,0,0,7.txt,0.495668,0.225470,0.039093,2,0
7,8,1989,,1958,M,79,1,1,0,0,1,0,0,8.txt,0.450710,0.172856,0.067859,0,0
8,9,2003,,1887,S,72,0,1,1,0,1,0,0,9.txt,0.506851,0.186567,0.037694,2,0
9,10,2013,,1990,R,68,8,1,1,0,1,1,0,10.txt,0.469250,0.213560,0.049106,9,0


**Анализ и визуализация**

In [42]:
!pip3 install pandas-profiling==2.11
from pandas_profiling import ProfileReport

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
profile = ProfileReport(df,
                        title='Pandas Profiling Report')
profile.to_widgets()

profile

/usr/local/lib/python3.8/dist-packages/pandas_profiling/profile_report.py:423: UserWarning: Ipywidgets is not yet fully supported on Google Colab (https://github.com/googlecolab/colabtools/issues/60).As an alternative, you can use the HTML report. See the documentation for more information.
  warnings.warn(


Summarize dataset:   0%|          | 0/32 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]